In [14]:
from tensorflow import keras
from tensorflow.keras.utils import plot_model
from keras.applications.inception_v3 import InceptionV3
from glob import glob
from keras.layers import Flatten,Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator,img_to_array,array_to_img,load_img

In [15]:
IMAGE_SIZE = [224,224]

In [16]:
# Create InceptionV3 Layer
inception = InceptionV3(include_top=False,weights='imagenet',input_shape=IMAGE_SIZE + [3])

In [17]:
for layer in inception.layers:
  layer.trainable = False

In [18]:
# we will use the glob to see how many outputs / labels there are
folders = glob('D:/Chrome Downloads/Tomato/tomato/train/*')

In [19]:
folders

['D:/Chrome Downloads/Tomato/tomato/train\\Tomato___Bacterial_spot',
 'D:/Chrome Downloads/Tomato/tomato/train\\Tomato___Early_blight',
 'D:/Chrome Downloads/Tomato/tomato/train\\Tomato___healthy',
 'D:/Chrome Downloads/Tomato/tomato/train\\Tomato___Late_blight',
 'D:/Chrome Downloads/Tomato/tomato/train\\Tomato___Leaf_Mold',
 'D:/Chrome Downloads/Tomato/tomato/train\\Tomato___Septoria_leaf_spot',
 'D:/Chrome Downloads/Tomato/tomato/train\\Tomato___Spider_mites Two-spotted_spider_mite',
 'D:/Chrome Downloads/Tomato/tomato/train\\Tomato___Target_Spot',
 'D:/Chrome Downloads/Tomato/tomato/train\\Tomato___Tomato_mosaic_virus',
 'D:/Chrome Downloads/Tomato/tomato/train\\Tomato___Tomato_Yellow_Leaf_Curl_Virus']

In [20]:


# Add Flatten and Dense in last layers
x = Flatten()(inception.output)
prediction = Dense(len(folders),activation='softmax')(x)

In [21]:
# create a model with additional flatten and dense layers and then we plot it
model = Model(inputs=inception.input,outputs = prediction)
plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [22]:
# Compile the model that we have created with adam and the loss is categorical_crossentropy because the classification is more than 2 classes
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [23]:
# Image Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2)
test_datagen = ImageDataGenerator(
    rescale=1./255)


In [27]:
# Apply Image Augmentation to Train and Test Images
training_set = train_datagen.flow_from_directory('D:/Chrome Downloads/Tomato/tomato/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = train_datagen.flow_from_directory('D:/Chrome Downloads/Tomato/tomato/val',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 10000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [28]:
import tensorflow as tf
# CallBack EarlyStop
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

In [33]:
# Train data
history = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=10,
    steps_per_epoch=218,
    validation_steps=len(test_set),
    callbacks=[early]
)

C:\Users\vinay\AppData\Local\Temp\ipykernel_11792\2876287022.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10
218/218 [==============================] - 181s 830ms/step - loss: 1.5237 - accuracy: 0.8908 - val_loss: 4.6389 - val_accuracy: 0.8080
Epoch 2/10
218/218 [==============================] - 182s 836ms/step - loss: 1.6799 - accuracy: 0.8963 - val_loss: 6.0529 - val_accuracy: 0.7350
Epoch 3/10
218/218 [==============================] - 180s 825ms/step - loss: 1.3636 - accuracy: 0.9019 - val_loss: 5.7559 - val_accuracy: 0.7660
Epoch 4/10
218/218 [==============================] - 180s 823ms/step - loss: 1.5556 - accuracy: 0.9019 - val_loss: 4.3139 - val_accuracy: 0.8100
Epoch 5/10
218/218 [==============================] - 179s 821ms/step - loss: 1.3328 - accuracy: 0.9061 - val_loss: 3.3794 - val_accuracy: 0.8250
Epoch 6/10
218/218 [==============================] - 179s 821ms/step - loss: 1.4508 - accuracy: 0.9055 - val_loss: 4.8184 - val_accuracy: 0.8020
Epoch 7/10
218/218 [==============================] - 179s 821ms/step - loss: 1.2925 - accuracy: 0.9151 - val_loss: 4.5874 -

In [34]:
# Model Evaluation
model.evaluate(test_set)

32/32 [==============================] - 22s 688ms/step - loss: 4.5937 - accuracy: 0.8080


[4.593689918518066, 0.8080000281333923]